In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
train_users = pd.read_csv('train_users.csv', header=0)
test_users = pd.read_csv('test_users.csv', header=0)

In [3]:
train_users["age"][train_users["age"] > 100] = np.NaN

/Users/CL/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [4]:
average_age_train = int(train_users["age"].mean())
average_age_train

36

In [5]:
train_users = train_users.fillna({'age': 'unknown'})

In [6]:
###Look at age for test set
#remove those over 100
test_users["age"][test_users["age"] > 100] = np.NaN

/Users/CL/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [7]:
average_age_test = test_users["age"].mean()
average_age_test

34.165688212927755

In [8]:
#put average ages in null values:
test_users = test_users.fillna({'age': 'unknown'})

In [9]:
#Divide gender into male and female
i = 0
def get_gender(gender):
    global i
    if gender != 'FEMALE' and gender != 'MALE':
        return 'FEMALE' if(i % 2) else 'MALE'
    i = i + 1
    return gender

# replace all values other than 'FEMALE' and 'MALE'
train_users['gender'] = train_users['gender'].apply(get_gender)
test_users['gender']   = test_users['gender'].apply(get_gender)

In [10]:
#divide date_account_created into year and month
# date_first_booking

def year(date):
    if date == date: 
        return int(str(date)[:4])
    return date

def month(date):
    if date == date: 
        return int(str(date)[5:7])
    return date

# Create Year and Month columns for account created
train_users['account_year']  = train_users['date_account_created'].apply(year)
train_users['account_month']  = train_users['date_account_created'].apply(month)

# convert type to integer
train_users[['account_year', 'account_month']] = train_users[['account_year', 'account_month']].astype(int)

#Do the same for test set:
test_users['account_year']  = test_users['date_account_created'].apply(year)
test_users['account_month']  = test_users['date_account_created'].apply(month)

# convert type to integer
test_users[['account_year', 'account_month']] = test_users[['account_year', 'account_month']].astype(int)

In [11]:
#remove date_account_created
train_users = train_users.drop(['date_account_created', 'date_first_booking'], axis=1)
test_users = test_users.drop(['date_account_created', 'date_first_booking'], axis=1)

In [12]:
train_users = train_users.fillna({'first_affiliate_tracked': 'unknown'})
test_users = test_users.fillna({'first_affiliate_tracked': 'unknown'})

In [13]:
train_users.head()

,id,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,account_year,account_month
0,gxn3p5htnn,20090319043255,MALE,unknown,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,2010,6
1,820tgsjxq7,20090523174809,MALE,38,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,2011,5
2,4ft3gnwmtx,20090609231247,FEMALE,56,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,2010,9
3,bjjt8pjhuk,20091031060129,FEMALE,42,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,2011,12
4,87mebub9p4,20091208061105,FEMALE,41,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,2010,9


In [14]:
#assign all objects as integers:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [15]:
train_users = MultiColumnLabelEncoder(columns = ['age', 'gender','language', 'signup_method', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked',
                                  'signup_app', 'first_device_type', 'first_browser']).fit_transform(train_users)
test_users = MultiColumnLabelEncoder(columns = ['age', 'gender','language', 'signup_method', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked',
                                  'signup_app', 'first_device_type', 'first_browser']).fit_transform(test_users)

In [16]:
country_to_num = {'NDF': 0, 'US': 1, 'other': 2, 'FR': 3, 'IT': 4, 'GB': 5, 'ES': 6, 'CA': 7, 'DE': 8, 'NL': 9, 'AU': 10, 'PT': 11}
num_to_country = {y:x for x,y in country_to_num.items()}


In [17]:
X_train = train_users.drop(["country_destination", "id"],axis=1)
Y_train = train_users["country_destination"]
Y_train = Y_train.map(country_to_num)

In [18]:
X_test = test_users.drop(['id'], axis=1).copy()

In [19]:
#Cross Validation
from sklearn import cross_validation

In [20]:
#Cross validation on train_set
Xcross_train, Xcross_test, ycross_train, ycross_test = cross_validation.train_test_split(
    X_train, Y_train, test_size=0.2, random_state=0)

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
clf = RandomForestClassifier(n_estimators=200,
    min_samples_split=4,
    min_samples_leaf=2)

In [23]:
clf = clf.fit(Xcross_train, ycross_train)

In [24]:
clf.score(Xcross_test, ycross_test)

0.62458129348106162

In [25]:
from sklearn.ensemble import AdaBoostClassifier

In [26]:
clf = AdaBoostClassifier(n_estimators=300)
clf = clf.fit(Xcross_train, ycross_train)

In [27]:
clf.score(Xcross_test, ycross_test)

0.63071841840200515

In [28]:
from sklearn import linear_model
logistic = linear_model.LogisticRegression()

In [29]:
clf = logistic.fit(Xcross_train, ycross_train)

In [30]:
clf.score(Xcross_test, ycross_test)

0.58672788175493662

In [31]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [32]:
clf = gnb.fit(Xcross_train, ycross_train)

In [33]:
clf.score(Xcross_test, ycross_test)

0.58269892951675994

In [34]:
from sklearn.ensemble import ExtraTreesClassifier

In [35]:
clf = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=1, random_state=0)

In [36]:
clf = clf.fit(Xcross_train, ycross_train)

In [37]:
clf.score(Xcross_test, ycross_test)

0.56044599564310982